# <strong>Proceso:</strong> Prueba Yapo
## <strong>Objetivo:</strong>
##### Generar un archivo Json con las series de tiempo por producto, tomando en cuenta la fecha y la cantidad de productos vendidos ese día.

## <h2>Descargas e Importaciones Iniciales:</h2>

In [ ]:
#Establece conexión a cuenta de google drive que contiene la data input
from google.colab import drive
drive.mount('/content/drive')


#Descarga de java jdk
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Descarga Apache Spark
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#Descomprime Tar
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#Instala Findspark
!pip install -q findspark

from datetime import datetime
import json
import os
#Definine Variables de Entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

Mounted at /content/drive


### Inicio Sesión de Spark

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

### Variables

In [ ]:
main_path = "drive/MyDrive/Prueba Yapo/"

folders = ["may", "june"]

#Se crea diccionario con key "", el cual sera eliminado previo a exportar el archivo json
data_dict = {"": {}}

### Unión de todos los CSV en un solo Dataframe

In [ ]:
for folder in folders:
  month_files = os.listdir(main_path+folder)
  #Lectura del CSV que contiene los headers
  df_eg = spark.read.options(header='True', inferSchema='True', delimiter=',')\
               .csv(main_path+folder+"/products00.csv")
  
  mySchema = df_eg.schema
  #Elimina CSV que ya fue leído de la lista de CSV por leer
  month_files = month_files[1:]

  #Construye rutas completas (path) de los csv para su lectura
  for position in range(0, len(month_files)):
    month_files[position] = main_path+folder+"/"+month_files[position]

  #Lee la lista de CSV pendientes
  month_df = spark.read.options(delimiter=',')\
                  .schema(mySchema)\
                  .csv(month_files)

  if folder == folders[0]:
    month_df_all = month_df.unionByName(df_eg)
  else:
    month_df_aux = month_df.unionByName(df_eg)
    month_df_all = month_df_all.unionByName(month_df_aux)


### Genera Dataframe con la información a escribir

In [ ]:
month_df_all = month_df_all.withColumn("creation_date", to_date(col("creation_date"), "yyyy-MM-dd"))\
                           .groupBy("product_name", "creation_date")\
                           .agg(count(col("product_name")).alias("num_of_sales"))\
                           .sort("product_name", "creation_date")

month_df_all = month_df_all.persist()

### Da formato a Dataframe para su posterior escritura en Json

In [ ]:
product_list = month_df_all.select("product_name").distinct().collect()

for product in product_list:
  
  date_list = month_df_all.filter(col("product_name")==product.product_name).select("creation_date").collect()
  sales_list = month_df_all.filter(col("product_name")==product.product_name).select("num_of_sales").collect()
  for elem in range(0, len(date_list) ):
    if elem == 0:
      data_dict[product.product_name] = {}
    data_dict[product.product_name][date_list[elem].creation_date.strftime("%Y-%m-%d")] = sales_list[elem].num_of_sales

#Elimina key ""
del data_dict[""]

## Escritura de Archivo Json

In [ ]:
with open(main_path+"ventas.json", "w") as output:
  json.dump(data_dict, output, indent=6)